<center><img src='https://raw.githubusercontent.com/language-ml/1-nlp-exploring-datasets/main/notebooks/images/besm.png' width='150px'><center/>

<center> درس پردازش زبان‌های طبیعی </center>
<center> آزمایشگاه پردازش هوشمند متن و زبان و علوم انسانی محاسباتی </center>

<h1>
<center>
پیش‌پردازش و بررسی متن فارسی
</center>
</h1>
<br/>
<h2>
<div dir='rtl'>مرتضی ابوالقاسمی ۴۰۰۲۱۰۹۲۵</div>
</h2>


<a href="https://colab.research.google.com/github/abreza/nlp_hw2/blob/main/nlp_hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div dir='rtl'>برای این تمرین بنده حدود ۱۳ کانال تلگرام را کرول کردم و اطلاعات استخراج شده را برای دسته‌بندی موضوعی استفاده خواهم کرد.

<br/>
دسته کانال‌های کرول شده عبارت‌اند از:

*   اقتصادی
  *   NavadeEghtesadi
  *   Eghtesadnews_com
  *   eghtesadonline
*   پزشکی
  *   MajalePezeshki
  *   majale_pzshki
  *   world_he
*   اخبار فن‌آوری
  *   thezoomit
  *   Digiato
  *   AllApk
  *   IRCOM
*   ادبیات
  *   sherbaranast2
  *   DivankadeAdabFarsi
  *   asharenab
</div>

##__ایمپورت‌ها </center>__

In [ ]:
!pip install hazm
!pip install parsivar
from IPython.display import clear_output 
clear_output()

In [ ]:
from IPython.display import clear_output 
import re, string, unicodedata, random, enum
import numpy as np
from collections import Counter

from hazm import Normalizer as HazmNormalizer, word_tokenize, Stemmer, Lemmatizer
from parsivar import Normalizer as ParsivarNormalizer

import nltk
nltk.download('punkt')
clear_output()

import pandas as pd
import codecs
import tqdm

import itertools

##__دانلود متن‌ها </center>__

In [ ]:
!wget -O files.zip https://github.com/abreza/nlp_hw2/blob/main/text.zip?raw=1
!wget -O stopwords.txt https://github.com/language-ml/1-nlp-exploring-datasets/blob/main/notebooks/farsi/stopwords.txt?raw=1
!unzip -o files.zip -d files/
!rm files.zip

clear_output()

##__استخراج متن‌ها</center>__

In [ ]:
def getPosts(files):
  posts = []
  posts_index = -1
  for cat in files:
    for fileName in files[cat]:
      with open('/content/files/' + fileName + '.txt') as file:
        lines = file.readlines()
        posts.append({
          'text': '',
          'cat': cat,
          'fileName': fileName
        })
        posts_index += 1
        for line in lines:
          if '#############' in line:
            posts.append({
              'text': '',
              'cat': cat,
              'fileName': fileName
            })
            posts_index += 1
            continue
          posts[posts_index]['text'] += line
  return posts

In [ ]:
class Category(enum.Enum):
  ECO = 1
  HEALTH = 2
  TECH = 3
  LITERATURE = 4

files = {}

files[Category.ECO] = ['NavadeEghtesadi', 'Eghtesadnews_com', 'eghtesadonline']
files[Category.HEALTH] = ['MajalePezeshki', 'majale_pzshki', 'world_he']
files[Category.TECH] = ['AllApk', 'Digiato', 'IRCOM', 'thezoomit']
files[Category.LITERATURE] = ['sherbaranast2', 'DivankadeAdabFarsi', 'asharenab']

posts = getPosts(files)

##__حذف اسامی کانال‌ها، لینک‌ها و ایموجی‌ها</center>__


<div dir='rtl'>حذف لینک‌های کانال تلگرامی</div>

In [ ]:
def removeTelegramWords(post):
  post['text'] = re.sub(r'@\S+', '', post['text'])
  post['text'] = re.sub(r'telegram.me\S+', '', post['text'])
  post['text'] = re.sub(r't.me\S+', '', post['text'])
  post['text'] = re.sub(r'\bChannel\b', '', post['text'])
  post['text'] = re.sub(r'\bکانال\b', '', post['text'])
  post['text'] = re.sub(r'\bJoin\b', '', post['text'])

<div dir='rtl'>
حذف لینک‌ها
</div>

In [ ]:
def removeLink(post):
  post['text'] = re.sub(r"http\S+", "", post['text'])

<div dir='rtl'>
حذف کاراکتر‌های خاص و ایموجی‌ها
</div>

In [ ]:
emoji_pattern = re.compile("["
  u"\U0001F600-\U0001F64F"  # emoticons
  u"\U0001F300-\U0001F5FF"  # symbols & pictographs
  u"\U0001F680-\U0001F6FF"  # transport & map symbols
  u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
  u"\U00002500-\U00002BEF"  # chinese char
  u"\U00002702-\U000027B0"
  u"\U00002702-\U000027B0"
  u"\U000024C2-\U0001F251"
  u"\U0001f926-\U0001f937"
  u"\U00010000-\U0010ffff"
  u"\u2640-\u2642" 
  u"\u2600-\u2B55"
  u"\u200d"
  u"\u23cf"
  u"\u23e9"
  u"\u231a"
  u"\ufe0f"  # dingbats
  u"\u3030" # flags (iOS)
"]+", flags=re.UNICODE)

def removeEmojiesAndSymbols(post):
  post['text'] = emoji_pattern.sub(r'', post['text'])
  post['text'] = re.sub('[!%#$,.`~!^&*()-+={}[]|\\//:;\"\'<,>?]؛', '', post['text'])

In [ ]:
for post in posts:
  removeTelegramWords(post)
  removeLink(post)
  removeEmojiesAndSymbols(post)

In [ ]:
for x in random.sample(posts, 30):
  print(x)

{'text': 'بازخوانی گزارش بازار: اپلیکیشن\u200cهای آموزشی چگونه در نبود حمایت\u200cهای دولتی رشد کردند\n\nبر اساس گزارشی که چندی پیش کافه_بازار منتشر کرد، اپلیکیشن\u200cهای آموزشی در سال ۱۳۹۹ در مقایسه با سال ۱۳۹۸ مورد اقبال بیشتری قرار گرفته و نصب آنها رشد قابل توجهی داشته است اما سوال اساسی این است که آیا این رشد حاصل حمایت\u200cهای دولتی است؟\n\nبنیان\u200cگذاران و مدیران اپلیکیشن\u200cهای آموزشی بر این باورند که اساسا حمایتی از طرف دولت انجام نشده و از سویی دیگر حتی شاهد بی\u200cتوجهی ظرفیت\u200cهای اپلیکیشن\u200cهای این حوزه در جریان توسعه آموزش الکترونیکی کشوره بوده\u200cایم\n\nبنیان\u200cگذاران اپلیکیشن\u200cهای آموزشی چون «بیاموز»، «آرش و آنا» و همچنین لرنیت در گفت\u200cوگو با دیجیاتو به بررسی جزئیات چالش\u200cهای پیش\u200c روی اپلیکیشن\u200cهای آموزشی پرداختند مشروح این گزارش و دلایل رشد اپلیکیشن\u200cهای آموزشی آنلاین را در سایت دیجیاتو بخوانید\n\n\n\n\u200c  \n', 'cat': <Category.TECH: 3>, 'fileName': 'Digiato'}
{'text': 'جان فدا کردیم و یاران قدرِ ما نشناختند\n\nکور بادا دید

##__نرمال کردن متن</center>__


In [ ]:
def sub_alphabets(text):
  items = [
    (r"ٲ|ٱ|إ|ﺍ|أ", r"ا"),
    (r"ﺁ|آ", r"آ"),
    (r"ﺐ|ﺏ|ﺑ", r"ب"),
    (r"ﭖ|ﭗ|ﭙ|ﺒ|ﭘ", r"پ"),
    (r"ﭡ|ٺ|ٹ|ﭞ|ٿ|ټ|ﺕ|ﺗ|ﺖ|ﺘ", r"ت"),
    (r"ﺙ|ﺛ", r"ث"),
    (r"ﺝ|ڃ|ﺠ|ﺟ", r"ج"),
    (r"ڃ|ﭽ|ﭼ", r"چ"),
    (r"ﺢ|ﺤ|څ|ځ|ﺣ", r"ح"),
    (r"ﺥ|ﺦ|ﺨ|ﺧ", r"خ"),
    (r"ڏ|ډ|ﺪ|ﺩ", r"د"),
    (r"ﺙ|ﺛ", r"ث"),
    (r"ﺫ|ﺬ|ﻧ", r"ذ"),
    (r"ڙ|ڗ|ڒ|ڑ|ڕ|ﺭ|ﺮ", r"ر"),
    (r"ﺰ|ﺯ", r"ز"),
    (r"ﮊ", r"ژ"),
    (r"ݭ|ݜ|ﺱ|ﺲ|ښ|ﺴ|ﺳ", r"س"),
    (r"ﺵ|ﺶ|ﺸ|ﺷ", r"ش"),
    (r"ﺺ|ﺼ|ﺻ", r"ص"),
    (r"ﺽ|ﺾ|ﺿ|ﻀ", r"ض"),
    (r"ﻁ|ﻂ|ﻃ|ﻄ", r"ط"),
    (r"ﻆ|ﻇ|ﻈ", r"ظ"),
    (r"ڠ|ﻉ|ﻊ|ﻋ", r"ع"),
    (r"ﻎ|ۼ|ﻍ|ﻐ|ﻏ", r"غ"),
    (r"ﻒ|ﻑ|ﻔ|ﻓ", r"ف"),
    (r"ﻕ|ڤ|ﻖ|ﻗ", r"ق"),
    (r"ڭ|ﻚ|ﮎ|ﻜ|ﮏ|ګ|ﻛ|ﮑ|ﮐ|ڪ|ك", r"ک"),
    (r"ﮚ|ﮒ|ﮓ|ﮕ|ﮔ", r"گ"),
    (r"ﻝ|ﻞ|ﻠ|ڵ", r"ل"),
    (r"ﻡ|ﻤ|ﻢ|ﻣ", r"م"),
    (r"ڼ|ﻦ|ﻥ|ﻨ", r"ن"),
    (r"ވ|ﯙ|ۈ|ۋ|ﺆ|ۊ|ۇ|ۏ|ۅ|ۉ|ﻭ|ﻮ|ؤ", r"و"),
    (r"ﺔ|ﻬ|ھ|ﻩ|ﻫ|ﻪ|ۀ|ە|ة|ہ", r"ه"),
    (r"ﭛ|ﻯ|ۍ|ﻰ|ﻱ|ﻲ|ں|ﻳ|ﻴ|ﯼ|ې|ﯽ|ﯾ|ﯿ|ێ|ے|ى|ي|یٰ", r"ی"),
    (r'¬|ۚ| ۖ| ۗ| ۚ| ۙ| ۘ| ۛ|۩|۞|↙|«|»', r'‌'),
    (r':|;|«|؛|!|؟|٪', r'‌'),

    (r'ـ|ِ|ُ|َ|ٍ|ٌ|ً', ''),
    
    (r'•|·|●|·|・|∙|｡|ⴰ', r''),
    (r',|٬|٫|‚|，|،|،', r''),
    (r'¬', r''),
    (r'( )+', r' '),
    (r'(\n)+', r'\n'),
    (r'(‌)+', r'‌'),
    (u"\ufdf0", r""),
    (u"\ufdf1", r""),
    (u"\u2022", r""),
  ]
  for item in items:
    text = re.sub(item[0], item[1], text)
  return text

In [ ]:
pNormalizer = ParsivarNormalizer()
hNormalizer = HazmNormalizer()

def normalizeText(text):
  text = sub_alphabets(text)
  text = pNormalizer.normalize(text)
  return hNormalizer.normalize(text)

In [ ]:
for post in posts:
  post['text'] = normalizeText(post['text'])

##__توکنایز کردن کلمات</center>__


In [ ]:
for post in posts:
  post['words'] = word_tokenize(post['text'])

##__حذف کلمات پرتکرار فارسی</center>__


In [ ]:
stopwords = ['_', '/', '-', ';', '،', '', '‌']
with open('/content/stopwords.txt') as file:
  stopwords += [normalizeText(line.split('\n')[0]) for line in file.readlines()]
print(stopwords)

['_', '/', '-', ';', '،', '', '\u200c', 'و', 'در', 'به', 'از', 'که', 'این', 'را', 'با', 'است', 'برای', 'آن', 'یک', 'خود', 'تا', 'کرد', 'بر', 'هم', 'نیز', 'گفت', 'می\u200cشود', 'وی', 'شد', 'دارد', 'ما', 'اما', 'یا', 'شده', 'باید', 'هر', 'آن\u200cها', 'بود', 'او', 'دیگر', 'دو', 'مورد', 'می\u200cکند', 'شود', 'کند', 'وجود', 'بین', 'پیش', '\u200cشده\u200c_ است', 'پس', 'نظر', 'اگر', 'همه', 'یکی', 'حال', 'هستند', 'من', 'کنند', 'نیست', 'باشد', 'چه', 'بی', 'می', 'بخش', 'می\u200cکنند', 'همین', 'افزود', 'هایی', 'دارند', 'راه', 'همچنین', 'روی', 'داد', 'بیشتر', 'بسیار', 'سه', 'داشت', 'چند', 'سوی', 'تنها', 'هیچ', 'میان', 'اینکه', 'شدن', 'بعد', 'جدید', 'ولی', 'حتی', 'کردن', 'برخی', 'کردند', 'می\u200cدهد', 'اول', 'نه', 'کرده _ است', 'نسبت', 'بیش', 'شما', 'چنین', 'طور', 'افراد', 'تمام', 'درباره', 'بار', 'بسیاری', 'می\u200cتواند', 'کرده', 'چون', 'ندارد', 'دوم', 'بزرگ', 'طی', 'حدود', 'همان', 'بدون', 'البته', 'آنان', 'می\u200cگوید', 'دیگری', 'خواهد _ شد', 'کنیم', 'قابل', 'یعنی', 'رشد', 'می\u200cتوان', 'وا

In [ ]:
for post in posts:
  post['notStopWords'] = [w for w in post['words'] if w not in stopwords]

<div dir='rtl'>حذف کلمات پرتکرار هر کانال</div>

In [ ]:
tokens = {}
for cat in files:
  tokens[cat] = []
  for fileName in files[cat]:
    filePosts = list(filter(lambda p: p['fileName'] == fileName, posts))
    fileWords = []
    for post in filePosts:
      fileWords += post["notStopWords"]
    tokens[cat] += fileWords

##__تحلیل بسامد</center>__

In [ ]:
df = {}

for cat in tokens:
  df[cat] = Counter(tokens[cat]).most_common(50)

freq_analysis = pd.DataFrame(df)

In [ ]:
freq_analysis

,Category.ECO,Category.HEALTH,Category.TECH,Category.LITERATURE
0,"(روز, 395)","(پزشکی, 260)","(سال, 248)","(دل, 360)"
1,"(قیمت, 378)","(کرونا, 250)","(شرکت, 237)","(ز, 356)"
2,"(ایران, 364)","(مجله, 217)","(استفاده, 219)","(عشق, 222)"
3,"(بازار, 364)","(واکسن, 156)","(گوشی, 197)","(سر, 197)"
4,"(هزار, 359)","(کاهش, 149)","(کاربران, 193)","(جان, 159)"
5,"(اقتصاد, 343)","(مصرف, 139)","(گلکسی, 184)","(دست, 145)"
6,"(دلار, 333)","(افزایش, 139)","(اپل, 179)","(دوست, 138)"
7,"(کشور, 316)","(بدن, 126)","(اطلاعات, 173)","(مرا, 128)"
8,"(دولت, 315)","(روز, 99)","(قرار, 159)","(چشم, 123)"
9,"(سال, 265)","(بیماری, 98)","(بازار, 140)","(چو, 115)"


In [ ]:
all_tokens = sum(tokens.values(), [])

print ('%-16s' % 'Number of words', '%-16s' % len(all_tokens))
print ('%-16s' % 'Number of unique words', '%-16s' % len(set(all_tokens)))
avg=np.sum([len(word) for word in all_tokens])/len(all_tokens)
print ('%-16s' % 'Average word length', '%-16s' % avg)
print ('%-16s' % 'Longest word', '%-16s' % all_tokens[np.argmax([len(word) for word in all_tokens])])

Number of words  132288          
Number of unique words 23878           
Average word length 5.095224056603773
Longest word     قحط‌الرجال‌زده‌شده‌است‌


##__ریشه‌یابی کلمات</center>__

In [ ]:
stemmer = Stemmer()
lemmatizer = Lemmatizer()

df = {}

for cat in tokens:
  tokens[cat] = list(map(stemmer.stem, tokens[cat]))
  tokens[cat] = list(map(lemmatizer.lemmatize, tokens[cat]))
  df[cat] = Counter(tokens[cat]).most_common(50)

stemm_lemm_freq_analysis = pd.DataFrame(df)

In [ ]:
stemm_lemm_freq_analysis

,Category.ECO,Category.HEALTH,Category.TECH,Category.LITERATURE
0,"(اقتصاد, 673)","(کرونا, 307)","(گوش, 311)","(دل, 493)"
1,"(روز, 430)","(پزشک, 293)","(سال, 264)","(ز, 408)"
2,"(بازار, 403)","(بیمار, 265)","(کاربر, 261)","(عشق, 247)"
3,"(کشور, 378)","(مجله, 217)","(شرک, 237)","(سر, 225)"
4,"(قیم, 378)","(واکسن, 185)","(استفاده, 222)","(کرد#کن, 181)"
5,"(ایر, 364)","(کاه, 149)","(کرد#کن, 208)","(دوس, 163)"
6,"(هزار, 359)","(بدن, 144)","(شد#شو, 196)","(ج, 159)"
7,"(دلار, 350)","(مصرف, 142)","(گلکس, 184)","(دس, 158)"
8,"(دول, 315)","(افزا, 139)","(اطلاع, 183)","(روز, 144)"
9,"(درصد, 285)","(آب, 129)","(اپل, 182)","(چ, 141)"


<div dir='rtl'>
ریشه‌یابی به کمک hazm نتیجه خوبی ندارد ولی بدلیل نبود جایگزین مناسب از آن استفاده می‌کنیم.

البته با وجود مشکلات خروجی نسبتا مناسبی بدست آمد.
</div>

##__دیدن سیاق</center>__

In [ ]:
hafez_text = nltk.Text(all_tokens)
hafez_text.concordance('قیمت')
hafez_text.concordance('غذا')
hafez_text.concordance('اندروید')
hafez_text.concordance('درخت')

Displaying 25 of 483 matches:
یمی‌ها محصولاتشان یارانه دولتی تولید قیمت جهانی می‌فروشند اکبر مشهدی رئیس اتحا
رصد گران‌تر تولیدکنندگان نقشی افزایش قیمت مسئولان دولتی افزایش قیمت روز روز دس
قشی افزایش قیمت مسئولان دولتی افزایش قیمت روز روز دستوری پتروشیمی محصولات پلاس
تیکی گران مجتمع‌های پتروشیمی محصولات قیمت جهانی می‌فروشند پتروشیمی ارز حمایتی 
وشیمی بورس ایجاد رقابت کاذب پتروشیمی قیمت واقعی دست تولیدکنندگان پلاستیک می‌رس
یمی یارانه حمایتی دولتی تولید‌شده‌را قیمت پایین کشورهای بفروشند شرایطی تولیدکن
 ایران‌ فیلم دولت به‌طور اسمی افزایش قیمت ۴۰ درصدی لاستیک لغو لاستیک ۴۰ درصد گ
گیری نکردند برنج ۶۰ هزار تومانی دروغ قیمت کیلو برنج ۴۸ هزار تومان جمیل علیزاده
ده برنج طارم معطر بازار تهران حداکثر قیمت ۴۸ هزار تومان فروخته_می‌شود افت ۱۶ د
رقم تشکیل سرمایه ثابت ناخالص کشورمان قیمت ثابت سال پایه ۱۳۹۰ ۱۶۰ هزار ۷۶۰ میلی
وی مصوبه هفته انجمن تایر مبنی افزایش قیمت ۴۰ درصدی لاستیک بگیریم لاستیک مشمول 
شمول قیمت‌گذاری انجمن تایر حق افزایش قیمت محصولات اعمال شرکت تولیدکننده تایر خ
ز صورت‌های مالی بدهند 

##__بررسی زوج کلمات پرتکرار</center>__

In [ ]:
bigrams = {}
df = {}

for cat in tokens:
  print(nltk.bigrams(tokens[cat]))
  bigrams[cat] = list(nltk.bigrams(tokens[cat]))
  df[cat] = Counter(bigrams[cat]).most_common(50)

bigram_freq_analysis = pd.DataFrame(df)

<generator object bigrams at 0x7f2419aec3d0>
<generator object bigrams at 0x7f2419aec3d0>
<generator object bigrams at 0x7f2419aec3d0>
<generator object bigrams at 0x7f2419aec3d0>


In [ ]:
bigram_freq_analysis

,Category.ECO,Category.HEALTH,Category.TECH,Category.LITERATURE
0,"((اقتصاد, ایران), 214)","((مجله, پزشک), 217)","((زوم, خواند#خوان), 67)","((سل, ساوج), 34)"
1,"((اخبار, تحلیل), 210)","((واکسن, کرونا), 43)","((اطلاع, زوم), 62)","((دوس, دار), 29)"
2,"((تحلیل, روز), 210)","((بیمار, کرونا), 36)","((گلکس, S), 50)","((فاضل, نظر), 25)"
3,"((روز, اقتصاد), 210)","((دز, واکسن), 32)","((گوش, هوشمند), 49)","((هوشنگ, ابتهاج), 23)"
4,"((‌نود, اقتصاد), 162)","((مبتلا, کرونا), 31)","((طرح, صیان), 48)","((صائب, تبریز), 22)"
5,"((میلیارد, تو), 87)","((روز, گذشته), 30)","((فضا, مجاز), 37)","((پروین, اعتصام), 18)"
6,"((هزار, تو), 84)","((سیس, ایمن), 30)","((شبکه, اجتماع), 36)","((سر, کو), 16)"
7,"((اتاق, بازرگان), 68)","((آمار, مبتلا), 29)","((وزیر, ارتباط), 35)","((دل, ز), 16)"
8,"((ایران, فیل), 59)","((کووید, ۱۹), 28)","((گلکس, اس), 33)","((حسین, منزو), 16)"
9,"((بانک, مرکز), 59)","((کرونا, شناسا), 28)","((S, ۲۲), 33)","((ملک, الشعرا), 14)"


<div dir='rtl'>
با توجه به ریشه‌یابی ضعیف hazm کمی نتایج مناسب نیست ولی در کل به اطلاعات خوبی رسیدیم.

در مورد بخش ادبیات بیشتر اسامی شعرا آمده است که لزوما بد نیست و در ادامه جفت‌کلمات پرتکرار آن را خواهیم دید.
</div>